In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output

# Import the CRUD module from crud.py
from crud import AnimalShelter

# Data Manipulation / Model
import pandas as pd
import base64

###########################
# Data Manipulation / Model
###########################
# MongoDB credentials (hardcoded username and password)
username = "aacuser"
password = "1234"

# Initialize the AnimalShelter class with the correct credentials
shelter = AnimalShelter(username, password)

# Retrieve all data from the MongoDB animals collection
data = shelter.read({})  # The empty dictionary {} retrieves all records

# Convert the data into a DataFrame
df = pd.DataFrame.from_records(data)

# Remove the '_id' column to avoid issues with the DataTable
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

# Debugging: Print out the first few rows of the data retrieved
print(df.head())

# Load Grazioso Salvare logo
image_filename = 'Grazioso Salvare Logo.png'  # Path to the logo image file
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('ProjectTwoDashboard')

app.layout = html.Div([
    # Display the Grazioso Salvare logo at the top
    html.Img(src=f'data:image/png;base64,{encoded_image}', style={'width': '300px'}),
    html.Center(html.B(html.H1('CS-340 Dashboard - Tyler Daniels'))),
    html.Hr(),
    
    # Rescue Type Filter (Radio Items for specific rescue types)
    dcc.RadioItems(
        id='rescue-type-radio',
        options=[
            {'label': 'Water Rescue', 'value': 'Water Rescue'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
            {'label': 'Disaster Rescue or Individual Tracking', 'value': 'Disaster Rescue or Individual Tracking'},
            {'label': 'Reset', 'value': 'Reset'}
        ],
        value='Reset',  # Default value is Reset
        labelStyle={'display': 'inline-block'}
    ),
    
    html.Hr(),
    
    # Preferred Breeds Filter (Dropdown)
    dcc.Dropdown(
        id='breed-filter',
        options=[
            {'label': 'Labrador Retriever', 'value': 'Labrador Retriever'},
            {'label': 'German Shepherd', 'value': 'German Shepherd'},
            {'label': 'Golden Retriever', 'value': 'Golden Retriever'}
        ],
        placeholder="Select a Breed",
        multi=False
    ),
    
    html.Hr(),
    
    # DataTable with pagination, sorting, and row selection
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),  # Populating DataTable with MongoDB data
        sort_action="native",  # Enable sorting
        page_action="native",  # Enable pagination
        page_size=10,  # Display 10 rows per page
        row_selectable="single",  # Enable single row selection
        selected_rows=[0],  # Default to the first row
    ),
    
    html.Br(),
    html.Hr(),
    
    # Div for the Map (Geolocation chart)
    html.Div(id='map-id', className='col s12 m6'),
    
    html.Br(),
    
    # Div for the second chart (Pie Chart)
    html.Div(id='chart-id', className='col s12 m6'),
])

#############################################
# Interaction Between Components / Controller
#############################################

# Callback for filtering data based on the Rescue Type and Breed dropdowns
@app.callback(
    Output('datatable-id', 'data'),
    [Input('rescue-type-radio', 'value'),
     Input('breed-filter', 'value')]
)
def update_dashboard(rescue_type, breed):
    # Start with an empty query (no filter)
    query = {}

    # Apply Rescue Type filter
    if rescue_type and rescue_type != 'Reset':
        query['animal_type'] = 'Dog'  # Assuming only dogs are used for rescue
        if rescue_type == 'Water Rescue':
            query['breed'] = {'$in': ['Labrador Retriever']}
        elif rescue_type == 'Mountain or Wilderness Rescue':
            query['breed'] = {'$in': ['German Shepherd', 'Golden Retriever']}
        elif rescue_type == 'Disaster Rescue or Individual Tracking':
            query['breed'] = {'$in': ['German Shepherd', 'Labrador Retriever']}

    # Apply Breed filter
    if breed:
        query['breed'] = breed

    # Fetch filtered data from MongoDB
    filtered_data = shelter.read(query)

    # Convert to DataFrame and return it to DataTable
    df_filtered = pd.DataFrame.from_records(filtered_data)
    if '_id' in df_filtered.columns:
        df_filtered.drop(columns=['_id'], inplace=True)

    return df_filtered.to_dict('records')


# Callback to update the geo-location chart when a row is selected
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if viewData is None or index is None:
        return [html.P("No data available.")]
    
    dff = pd.DataFrame.from_dict(viewData)
    
    # Default to the first row if no row is selected
    if index is None:
        row = 0
    else:
        row = index[0]
        
    # Return Leaflet map with marker for the selected animal's location
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]], children=[
                dl.Tooltip(dff.iloc[row, 4]),  # Tooltip shows animal breed
                dl.Popup([html.H1("Animal Name"), html.P(dff.iloc[row, 9])])  # Popup shows animal name
            ])
        ])
    ]

# Callback to generate a second chart (e.g., Pie Chart)
@app.callback(
    Output('chart-id', 'children'),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_chart(viewData):
    if viewData is None:
        return [html.P("No data available.")]

    dff = pd.DataFrame.from_dict(viewData)
    
    # Generate a pie chart of animal outcomes by breed
    pie_chart = px.pie(dff, names='breed', title='Breed Distribution of Rescue Dogs')
    
    return [
        dcc.Graph(figure=pie_chart)
    ]

# Run the server to display the dashboard
app.run_server(debug=True, port=8080)

     age_upon_outcome animal_id animal_type  \
0  8           1 year   A736551         Dog   
1  1          3 years   A746874         Cat   
2  5          2 years   A691584         Dog   
3  3          2 years   A716330         Dog   
4  4         7 months   A733653         Cat   

                                      breed        color date_of_birth  \
0  Labrador Retriever/Australian Cattle Dog        Black    2015-10-12   
1                    Domestic Shorthair Mix  Black/White    2014-04-10   
2                    Labrador Retriever Mix    Tan/White    2012-11-06   
3                   Chihuahua Shorthair Mix  Brown/White    2013-11-18   
4                               Siamese Mix   Seal Point    2016-01-25   

              datetime            monthyear   name outcome_subtype  \
0  2016-11-27 18:00:00  2016-11-27T18:00:00   *Mia                   
1  2017-04-11 09:00:00  2017-04-11T09:00:00                   SCRP   
2  2015-05-30 13:48:00  2015-05-30T13:48:00   Luke            